## PydanticOutputParser

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [2]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [ ]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [5]:
# 출력 파서를 사용하지 않을 경우에 대한 예시

from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)

중요 내용 요약:

1. 발신자: 김철수 (바이크코퍼레이션 상무)
2. 수신자: 이은채 (Teddy International)
3. 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. 요청 사항:
   - ZENESIS 모델에 대한 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. 미팅 제안:
   - 날짜: 1월 15일 (다음 주 화요일)
   - 시간: 오전 10시
   - 장소: 귀사 사무실

6. 목적: 협력 가능성 논의 및 유통 전략, 마케팅 계획 구체화.

In [6]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [7]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [8]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [9]:
chain = prompt | llm

In [10]:
response = chain.stream(
    {
        "question": "이메일 내용 중 주요 내용을 추출해 주세요.",
        "email_conversation": email_conversation
    }
)

output = stream_response(response, return_output= True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무가 이은채 대리님에게 'ZENESIS' 자전거에 대한 브로슈어 요청과 함께 협력 가능성을 논의하기 위한 미팅을 제안함.",
  "date": "1월 15일 오전 10시"
}
```

In [11]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="김철수 상무가 이은채 대리님에게 'ZENESIS' 자전거에 대한 브로슈어 요청과 함께 협력 가능성을 논의하기 위한 미팅을 제안함." date='1월 15일 오전 10시'


In [ ]:
# 출력 파서를 추가하여 전체 체인을 재구성; 아예 처음부터 이렇게 parser를 때려버려도 되긴 함
chain = prompt | llm | parser

In [ ]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response


In [13]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini"
).with_structured_output(EmailSummary)


In [14]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='ZENESIS 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 'ZENESIS' 유통 협력에 대해 논의하고자 미팅을 제안하며, 제품 브로슈어 요청.", date='2024-01-15T10:00:00')

## 콤마 구분자 출력 파서

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [17]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

In [18]:
# "대한민국 관광명소"에 대한 체인 호출 실행
chain.invoke({"subject": "대한민국 관광명소"})

['경복궁', '인사동', '부산 해운대해수욕장', '제주도', '남산타워']

## 구조화된 출력 파서

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [21]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# 사용자의 질문에 대한 답변의 형식; dict 구조로 반환되는데, key 값은 answer, 그리고 source이다. source의 형식은 출처 혹은 웹사이트 주소로 지정해 두었다.
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [23]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the user's question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [24]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [25]:
# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [ ]:
for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화 발전에 큰 기여를 한 업적이 있습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}


## JSON 출력 파서

In [28]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [31]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [32]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명") # 말 그대로 description
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)") # #공부, #운동 등

In [33]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic) # Topic json 모양에 맞게 출력한다


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions()) # format은 parser에서 정의한 json 구조 바탕

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 평균 기온 상승, 해수면 상승, 극단적인 기상 현상을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화'}

In [34]:
# 질의 작성; 여기서 차이가 나는데, 그냥 json output parser를 사용하면서, 프롬프트에 어떻게 구성할지 작성한다~
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)


{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 파괴 등의 문제가 발생할 수 있습니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구환경']}


## 데이터 프레임 파서

In [42]:
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [44]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [45]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [46]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [47]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [48]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [ ]:
# 열 작업 예시입니다.
df_query = "Age"


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)


In [ ]:
# 행 조회 예시입니다.
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)


In [55]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()

31.2

In [ ]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

In [ ]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Calculate average `Fare` rate."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

In [58]:
# 결과 검증
df["Fare"].mean()

22.19937